In [1]:
import torch
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.nn import Sequential,MaxPool2d,Linear,ReLU
from functions_pytorch import TripletGenerator,loss_func_generator,l2_distance,SpatialTransformerNet
from pytorch_util import fine_tune_pretrainedmodels,fit,predict,trainable_parameter,set_requires_grad
from pytorch_models import ConvBatchRelu
from albumentations_helper import create_transform
from albumentations import ShiftScaleRotate,Cutout,RandomContrast,RandomBrightness,Compose
import pickle
import numpy as np
import pretrainedmodels

In [2]:
## configs ##
color = True
#shapes = (3,224,224)
HalfBatch = 8
margin = -2
outDim = 500
distanceFun = l2_distance

In [3]:
if color:
    with open('/home/will/Desktop/kaggle/Whale/train_df_color.pkl', 'rb') as f:
        Ids_train = pickle.load(f)
    with open('/home/will/Desktop/kaggle/Whale/new_whale_train_color.pkl', 'rb') as f:
        newWhale_train = pickle.load(f)
    with open('/home/will/Desktop/kaggle/Whale/val_df_color.pkl', 'rb') as f:
        Ids_val = pickle.load(f)
    with open('/home/will/Desktop/kaggle/Whale/new_whale_val_color.pkl', 'rb') as f:
        newWhale_val = pickle.load(f)
else:
    with open('/home/will/Desktop/kaggle/Whale/train_df.pkl', 'rb') as f:
        Ids_train = pickle.load(f)
    with open('/home/will/Desktop/kaggle/Whale/new_whale_train.pkl', 'rb') as f:
        newWhale_train = pickle.load(f)
    with open('/home/will/Desktop/kaggle/Whale/val_df.pkl', 'rb') as f:
        Ids_val = pickle.load(f)
    with open('/home/will/Desktop/kaggle/Whale/new_whale_val.pkl', 'rb') as f:
        newWhale_val = pickle.load(f)

Set up dataloader

In [4]:
aug = Compose([RandomContrast(p=0.5),RandomBrightness(p=0.5),
                ShiftScaleRotate(shift_limit=0.03,rotate_limit=25,scale_limit=0.05,p=1),Cutout(p=0.5)])
transform = create_transform(aug)  

aug_test = Compose([RandomContrast(p=0.2),RandomBrightness(p=0.2),
                ShiftScaleRotate(shift_limit=0.03,rotate_limit=15,scale_limit=0.02,p=1)])
transform_test = create_transform(aug_test)    

In [5]:
gen_train = TripletGenerator(Ids_train,newWhale_train,transform)
gen_val = TripletGenerator(Ids_val,newWhale_val,transform_test)        
#gen_train = FunctionWrapOverDataset(gen_train,numpy2torch)
#gen_val = FunctionWrapOverDataset(gen_val,numpy2torch)
train_dl= DataLoader(gen_train,HalfBatch,True,num_workers=3,drop_last=True)
valid_dl = DataLoader(gen_val,HalfBatch,False,num_workers=3,drop_last=True)

Set up training

In [6]:
convNet = Sequential(ConvBatchRelu(3,8,5,stride=2),\
                     ConvBatchRelu(8,16,5,stride=2),\
                     ConvBatchRelu(16,32,5,stride=2),\
                     MaxPool2d(2))

In [7]:
fcNet = Sequential(Linear(4608,32),ReLU(True),Linear(32,6))

In [8]:
with torch.no_grad():
    fcNet[2].weight.data.zero_()
    fcNet[2].bias.data.copy_(torch.tensor([1, 0, 0, 0, 1, 0], dtype=torch.float))

In [9]:
stn = SpatialTransformerNet(convNet,fcNet)

In [10]:
#base = pretrainedmodels.resnet50()
base = pretrainedmodels.densenet121()
base = fine_tune_pretrainedmodels(base, outDim)

In [11]:
model = Sequential(stn,base).to('cuda:0')

In [12]:
#model = pretrainedmodels.resnet50()
#model = fine_tune_pretrainedmodels(model, outDim).to('cuda:0')

In [12]:
loss_func = loss_func_generator(HalfBatch,margin,distanceFun)

In [13]:
opt = Adam(trainable_parameter(model),lr=1e-4)

In [14]:
model = fit(5, model, loss_func, opt, train_dl, valid_dl)

epoch:0, train_loss:-0.4914107208311558, val_loss:-1.0514062643051147
epoch:1, train_loss:-1.0556528090894222, val_loss:-1.340457797050476
epoch:2, train_loss:-1.1988435634732246, val_loss:-1.3510195016860962
epoch:3, train_loss:-1.3431288402557373, val_loss:-1.4981499910354614
epoch:4, train_loss:-1.4213895859956742, val_loss:-1.5577794313430786
Training completed in 303.7914445400238s


In [17]:
model = fit(5, model, loss_func, opt, train_dl, valid_dl)

epoch:0, train_loss:-1.4774936194181443, val_loss:-1.5426876544952393
epoch:1, train_loss:-1.4668679677009582, val_loss:-1.5668240785598755
epoch:2, train_loss:-1.547742312479019, val_loss:-1.5553418397903442
epoch:3, train_loss:-1.548053457069397, val_loss:-1.644281268119812
epoch:4, train_loss:-1.5597363849639894, val_loss:-1.6576347351074219
Training completed in 304.294842004776s


Fine-tune previous layers

In [ ]:
# for ResNet50
set_requires_grad([model[1].layer4,model[1].layer3,model[1].layer2],True)
opt = Adam([
            {"params": model[1].layer4.parameters(), "lr": 1e-5},
            {"params": model[1].layer3.parameters(), "lr": 1e-6},
            {"params": model[1].layer2.parameters(), "lr": 5e-7},
            {"params": model[1].last_linear.parameters(), "lr": 2e-5},
            {"params": model[0].parameters(), "lr": 2e-5}
            ])

In [55]:
# for DenseNet121
name_list = ['denseblock3','transition3','denseblock4','norm5']
set_requires_grad_paraList(gather_parameter_nameList(name_list,model),True)

In [56]:
opt = Adam([
            {"params": gather_parameter_nameList(['denseblock4','norm5'],base), "lr": 1e-5},
            {"params": gather_parameter_nameList(['denseblock3','transition3'],base), "lr": 1e-6},
            {"params": base.last_linear.parameters(), "lr": 2e-5},
            {"params": stn.parameters(), "lr": 2e-5}
            ])

In [57]:
model = fit(5, model, loss_func, opt, train_dl, valid_dl)

epoch:0, train_loss:-1.5874563907146453, val_loss:-1.6335901021957397
epoch:1, train_loss:-1.6483672627925874, val_loss:-1.7336546182632446
epoch:2, train_loss:-1.6909746215343475, val_loss:-1.7607063055038452
epoch:3, train_loss:-1.7239739263534546, val_loss:-1.734588861465454
epoch:4, train_loss:-1.7105337745189666, val_loss:-1.7588655948638916
Training completed in 358.9573745727539s
